##Importing libraries

In [102]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import xgboost as xgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

##Loading dataset

In [58]:
df = pd.read_csv('/content/marketing_campaign.csv',delimiter = '\t')

In [63]:
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,member_since,Age,TotalFamilyMembers,label
0,5524,1957,Graduation,1,58138.0,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,3347,64,1,6
1,2174,1954,Graduation,1,46344.0,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2797,67,3,0
2,4141,1965,Graduation,2,71613.0,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2996,56,2,0
3,6182,1984,Graduation,2,26646.0,1,0,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2823,37,3,0
4,5324,1981,PhD,2,58293.0,1,0,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,2845,40,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,10870,1967,Graduation,2,61223.0,0,1,13-06-2013,46,709,43,182,42,118,247,2,9,3,4,5,0,0,0,0,0,0,3,11,0,3065,54,3,0
2211,4001,1946,PhD,2,64014.0,2,1,10-06-2014,56,406,0,30,0,0,8,7,8,2,5,7,0,0,0,1,0,0,3,11,0,2703,75,5,1
2212,7270,1981,Graduation,1,56981.0,0,0,25-01-2014,91,908,48,217,32,12,24,1,2,3,13,6,0,1,0,0,0,0,3,11,0,2839,40,1,4
2213,8235,1956,Master,2,69245.0,0,1,24-01-2014,8,428,30,214,80,30,61,2,6,5,10,3,0,0,0,0,0,0,3,11,0,2840,65,3,0


##Formatting Data

In [59]:
current_date = date(2021,11,3)
df['member_since']=pd.to_datetime(df['Dt_Customer'], dayfirst=True,format = '%d-%m-%Y')
df['member_since'] = pd.to_numeric(df['member_since'].dt.date.apply(lambda x: (current_date - x)).dt.days, downcast='integer')
df['Age']=2021-df['Year_Birth']
df['Marital_Status']=df['Marital_Status'].replace({'Divorced':1,'Single':1,'Married':2,'Together':2,'Absurd':1,'Widow':1,'YOLO':1, 'Alone':1})
df['TotalFamilyMembers'] = df['Marital_Status'].astype(int) + df['Kidhome'].astype(int) + df['Teenhome'].astype(int)
df.dropna(subset = ['Income'], inplace=True)
df=df[df['Income']<600000]
df.reset_index(inplace=True, drop = True)

In [41]:
df = df[['member_since', 'Age', 'TotalFamilyMembers', 'Income','Recency', 'MntWines', 'MntFruits','MntMeatProducts', 'MntFishProducts', 'MntSweetProducts','MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases','NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4', 'AcceptedCmp5', 'Complain', 'Response']]

In [67]:
X = df[['member_since', 'Age', 'TotalFamilyMembers', 'Income','Recency', 'MntWines', 'MntFruits','MntMeatProducts', 'MntFishProducts', 'MntSweetProducts','MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases','NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',  'Complain']]
y = df['label']

In [ ]:
df['label'] = 0
for i in range(len(df['Age'])):
  if int(df['AcceptedCmp1'][i]) == 1:
    df['label'][i] = 1
  elif int(df['AcceptedCmp2'][i]) == 1:
    df['label'][i] = 2
  elif int(df['AcceptedCmp3'][i]) == 1:
    df['label'][i] = 3
  elif int(df['AcceptedCmp4'][i]) == 1:
    df['label'][i] = 4
  elif int(df['AcceptedCmp5'][i]) == 1:
    df['label'][i] = 5
  elif int(df['Response'][i]) == 1:
    df['label'][i] = 6

#Splitting Data

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=5)

##Running the model

In [99]:
model = XGBClassifier(learning_rate=0.001, n_jobs=4, n_estimators=50, max_depth = 3, colsample_bytree = 0.95, colsample_bynode=0.9) 
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=0.9, colsample_bytree=0.95, gamma=0,
              learning_rate=0.001, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=4,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

##Predicting values

In [100]:
y_pred = model.predict(X_test) 
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions) 
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 68.89%
